<a href="https://colab.research.google.com/github/xiaoyongzhu/hackathon/blob/master/Tensortrade_Example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Install Tensortrade and set Tensorflow to v2




In [0]:
!pip install tensorflow-probability==0.7.0 git+https://github.com/tensortrade-org/tensortrade.git

# code via https://github.com/tensortrade-org/tensortrade/blob/master/examples/train_and_evaluate.ipynb
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf

import ssl
import pandas as pd
# Only used if pandas gives a SSLError when using fetch()
ssl._create_default_https_context = ssl._create_unverified_context 

# Load Custom Data

## Mount Google Drive

In [0]:
from google.colab import drive
drive.mount('/content/drive/')

%cd '/content/drive/My Drive/Colab_Notebooks/Crypto_Data/'
#!mkdir -p /content/drive/My\ Drive/Colab_Notebooks/
#!mkdir -p /content/drive/My\ Drive/Colab_Notebooks/Crypto_Data/

## Load Custom CSV Data

In [0]:
exchange = 'binance'
symbol = 'BTC_USDT' #only exceptions are futures: eg. XRPH20 has no underscore
tf = '1d'
path_to_csv_file = f'csv/{exchange}/{symbol}_{tf}.csv'

import os
if os.path.exists(path_to_csv_file):
    # Use Pandas to read the .csv file
    df = pd.read_csv(path_to_csv_file)
    # Format the dataframe columns and column names for TensorTrade
    df.drop(columns=['open_timestamp_utc','close_timestamp_utc'], inplace=True)
    df = df.rename({"timestamp": "Date"}, axis='columns')
    
    from datetime import datetime
    df['Date'] = df['Date'].apply(lambda x: datetime.utcfromtimestamp(x))
    df['Date'] = df['Date'].dt.strftime('%Y-%m-%d %I:%M %p')
    df.sort_values(by='Date', ascending=True, inplace=True)
    df = df.set_index("Date")

    # OHCLV for plotly chart
    # ohclv_df = df.copy()
    # ohclv_df = ohclv_df.rename({"Date": "datetime"}, axis='columns')
    # ohclv_df = ohclv_df.set_index("datetime")

    #df.reset_index(drop=True, inplace=True)
    
    base, quote = symbol.split('_')
    df.columns = [base + ":" + name.lower() for name in df.columns]
    print(df.head())
    #print(ohclv_df.head())

    coinbase_data = df.copy()

#Setup OHLCV Data Fetcher

In [0]:
# Fetch Crypto data
def fetch(exchange_name, symbol, timeframe):  
    url = "https://www.cryptodatadownload.com/cdd/"
    filename = "{}_{}USD_{}.csv".format(exchange_name, symbol, timeframe)
    volume_column = "Volume {}".format(symbol)
    new_volume_column = "Volume_{}".format(symbol)
    
    df = pd.read_csv(url + filename, skiprows=1)
    df = df[::-1]
    df = df.drop(["Symbol"], axis=1)
    df = df.rename({"Volume USD": "volume", volume_column: new_volume_column}, axis=1)
    df = df.set_index("Date")
    df.columns = [symbol + ":" + name.lower() for name in df.columns]

    return df

#Fetch Historical Data

In [0]:
coinbase_data = fetch("Coinbase", "BTC", "1h")

# horizontally concatenate 2 dataframes
# coinbase_data = pd.concat([
#                     fetch("Coinbase", "BTC", "1h"),
#                     fetch("Coinbase", "ETH", "1h")
#                 ], axis=1)

In [0]:
coinbase_data.head()

#Setup Trading Environment

In [0]:
from tensortrade.exchanges import Exchange
from tensortrade.instruments import USD, BTC, ETH
from tensortrade.wallets import Wallet, Portfolio

from tensortrade.data import Stream, DataFeed, Module
from tensortrade.environments import TradingEnvironment
from tensortrade.exchanges.services.execution.simulated import execute_order

#Define Exchanges
An exchange needs a name, an execution service, and streams of price data in order to function properly.

The setups supported right now are the simulated execution service using simulated or stochastic data. More execution services will be made available in the future, as well as price streams so that live data and execution can be supported.

In [0]:
coinbase = Exchange("coinbase", service=execute_order)(
             Stream("USD-BTC", list(coinbase_data['BTC:close']))#,
             #Stream("USD-ETH", list(coinbase_data['ETH:close']))
           )

Now that the exchanges have been defined we can define our features that we would like to include, excluding the prices we have provided for the exchanges.

#Define External Data Feed
Here we will define the external feed to use whatever data you would like. From financial indicators to datasets that have nothing to do with instrument prices, they will all have to be defined and incorporated into the external data feed provided to the environment. There is also an internal data feed that is used to collect data on all the wallets and the net worth of the portfolio. You can choose to include this or not by using the use_internal parameter of the trading environment.

In [0]:
coinbase_btc = coinbase_data.loc[:, [name.startswith("BTC") for name in coinbase_data.columns]]
#coinbase_eth = coinbase_data.loc[:, [name.startswith("ETH") for name in coinbase_data.columns]]

# Add TA features

In [0]:
!pip install ta
import ta

ta.add_all_ta_features(
    coinbase_btc,
    colprefix="BTC:",
    **{k: "BTC:" + k for k in ['open', 'high', 'low', 'close', 'volume']}
)

# Create a 'DataFeed' for the Gym TradingEnvironment to be able to observe

In [0]:
with Module("coinbase") as coinbase_ns:
    nodes = [Stream(name, list(coinbase_data[name])) for name in coinbase_data.columns]
    #nodes += [Stream(name, list(coinbase_eth[name])) for name in coinbase_eth.columns]

In [0]:
feed = DataFeed([coinbase_ns])
feed.next()

#Portfolio
Make the portfolio using the any combinations of exchanges and intruments that the exchange supports

In [0]:
portfolio = Portfolio(USD, [
    Wallet(coinbase, 100000 * USD),
    Wallet(coinbase, 0 * BTC)#,
    #Wallet(coinbase, 5 * ETH),
])

In [0]:
env = TradingEnvironment(
    feed=feed,
    portfolio=portfolio,
    action_scheme='managed-risk',
    reward_scheme='risk-adjusted',
    window_size=20#,
    # renderers='screenlog'#,
    # price_history=ohclv_df
)

#Example Data Feed Observation

In [0]:
env.feed.next()

#Setup and Train DQN Agent

In [0]:
from tensortrade.agents import DQNAgent

agent = DQNAgent(env)

agent.train(n_steps=50, save_path="agents/", n_episodes = 5)

#Plot Performance

In [0]:
portfolio.performance.plot()

In [0]:
portfolio.performance.net_worth.plot()